In [1]:
def count_words(filepath):
    freq = {}
    for piece in open(filepath).read().lower().split():
        # only consider alphabetic characters 
        word = ''.join(c for c in piece if c.isalpha())
        if word:
            freq[word] = 1 + freq.get(word, 0) # creates a word key with a default value of 0 & updates with 1 every time
            
    max_word = ''
    max_count = 0
    for word, count in freq.items():
        if count > max_count:
            max_word = word
            max_count = count
            
    print('The most frequent word is', max_word)
    print('Its number of occurences is', max_count)

In [2]:
filepath = r'C:\Users\HP Probook\Documents\Programming files\Python codes\Billboard\Rihanna.txt'

In [3]:
count_words(filepath=filepath)

The most frequent word is rihanna
Its number of occurences is 11


### MapBase

In [6]:
from collections.abc import MutableMapping
class MapBase(MutableMapping):
    """An abstract base class that includes a nonpublic _Item class"""
    #------------nested _Item class
    
    class _Item:
        """Lightweight composite to store key-value pairs as map items"""
        __slots__ = '_key', '_value'
        
        def __init__(self, k, v):
            self._key = k
            self._value = v
        
        def __eq__(self, other):
            return self._key == other._key # compares items based on their keys
        
        def __ne__(self, other):
            return not(self == other) # opposite of __eq__
        
        def __It__(self, other):
            return self._key < other._key # compare items based on their keys

In [17]:
class UnsortedTableMap(MapBase):
    """Map implementation using an unordered list"""
    
    def __init__(self):
        """Create an empty map"""
        self._table = [] # list of _Items
        
    def __getitem__(self, k):
        """Return value associated with key k (raise KeyError if not found)"""
        for item in self._table:
            if k == item._key:
                return item._value
        raise KeyError('Key Error: ' + repr(k))
        
    def __setitem__(self, k, v):
        """Asign value v to key k, overwriting existing value if present"""
        for item in self._table:
            if k == item._key: # found a match
                item._value = v
                return
        self._table.append(self._Item(k, v))
        
    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found)"""
        for j in range(len(self._table)):
            if k == self._table[j]._key: # found a match
                self._table.pop(j) # remove item
                return # and quit
        raise KeyError('Key Error: ' + repr(k))
        
    def __len__(self):
        """Return number of items in the map"""
        return len(self._table)
    
    def __iter__(self):
        """Generate iteration of the map's keys"""
        for item in self._table:
            yield item._key

In [22]:
x = UnsortedTableMap()

In [23]:
x.__setitem__(2, 5)

In [24]:
x.__len__()

1

In [25]:
x.__getitem__(2)

5

In [26]:
x.__setitem__(3, 20)
x.__setitem__('keyx', 54)

In [27]:
x.__len__()

3

In [29]:
f = x.__iter__()
list(f)

[2, 3, 'keyx']

In [30]:
x.__delitem__('keyx')

In [31]:
x.__getitem__('keyx')

KeyError: "Key Error: 'keyx'"

### Hash Tables

In [104]:
from random import randrange
class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression"""
    def __init__(self, cap=11, p=109345121):
        """Create an empty hash-table map"""
        self._table = cap *[None]
        self._n = 0 # number of entries in the map
        self._prime = p
        self._scale = 1 + randrange(p - 1) # scale from 1 to p - 1 for MAD
        self._shift = randrange(p) # shift from 0 to p-1 for MAD
        
    def _hash_function(self, k):
        return (hash(k) * self._scale + self._shift) % self._prime % len(self._table)
    
    def __len__(self):
        return self._n
    
    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j, k) # this may raise KeyError
    
    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)
        if self._n > len(self._table)//2: # keep load factor <= 0.5
            self._resize( 2 * len(self._table) - 1) # to resize to a prime number
            
    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)
        self._n -= 1
        
    def _resize(self, c):
        old = list(self.items())
        self._table = c*[None]
        self._n = 0 # n will be recomputed during subsequent adds
        for (k, v) in old:
            self[k] = v # reinsert old key-value pair

In [105]:
class ChainHashMap(HashMapBase):
    """Hash map implemented with separate chaining for collision resolution"""
    
    def _bucket_getitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k)) # no match found
        return bucket[k]
        
    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap() # creates an instance of a map, bucket is new to the table
        oldsize = len(self._table[j])
        self._table[j][k] = v
        if len(self._table[j]) > oldsize: # key is new to the table
            self._n += 1
            
    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k)) # no match found
        del bucket[k]
        
    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:
                for key in bucket:
                    yield key

In [106]:
y = ChainHashMap()

In [107]:
y.__len__()

0

In [108]:
y.__setitem__(2, 5)

In [109]:
y.__len__()

1

In [110]:
y.__setitem__(3, 20)

In [111]:
y.__setitem__('four', 44)

In [112]:
y.__len__()

3

In [113]:
y.__getitem__('four')

44

### Linear Probing

In [114]:
class ProbeHashMap(HashMapBase):
    """Hash map, implemented with linear probing for collision resolution"""
    _AVAIL = object() # sentinel marks locations of previous deletions
    
    def _is_available(self, j):
        """Return True if index j is available in table"""
        return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL
    
    def _find_slot(self, j, k):
        """Search for key k in bucket at index j
        
        Return (success, index) tuple described as follows:
        If match was found, success is True and index denotes its location
        If no match was found, success if False and index denotes first available slot"""
        
        firstAvail = None
        while True:
            if self._is_available(j):
                if firstAvail is None:
                    firstAvail = j
                if self._table[j] is None:
                    return (False, firstAvail)
            elif k == self._table[j]._key:
                return (True, j)
            j = (j + 1)%len(self._table) # keep looking cyclically
            
    def _bucket_getitem(self, j, k):
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError('Key Error: ' + repr(k))
        return self._table[s]._value
    
    def _bucket_setitem(self, j, k, v):
        found, s = self._find_slot(j, k)
        if not found:
            self._table[s] = self._Item(k, v) # insert new item
            self._n += 1 # increase size
        else:
            self._table[s]._value = v # overwrite existing
            
    def _bucket_delitem(self, j, k):
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError('Key Error: ' + repr(k))
        self._table[s] = ProbeHashMap._AVAIL # mark the spot as vacted
        
    def __iter__(self):
        for j in range(len(self._table)):
            if not self._is_available(j):
                yield self._table[j]._key

In [115]:
z = ProbeHashMap()

In [116]:
z.__len__()

0

In [117]:
z.__setitem__(2, 5)

In [118]:
z.__setitem__(3, 10)
z.__setitem__('three', 2)
z.__setitem__(34, 'Akorede')

In [119]:
z.__len__()

4

In [120]:
l = list(z.__iter__())
l

[3, 34, 'three', 2]

In [121]:
z.__delitem__('three')

In [122]:
z.__len__()

3

### Sorted Maps

In [123]:
class SortedTableMap(MapBase):
    """Map implemntation using a sorted table"""
    
    #-----------nonpublic behaviors ------------
    def _find_index(self, k, low, high):
        """Return index of the leftmost item with key greater than or equal to k
        
        Return high + 1 if no such item qualifies
        That is, j will be returned such that:
        all items of slice table[low:j] have key < k
        all items of slice table[j:high + 1] have key >= k
        """
        
        if high < low:
            return high + 1 # no element qualifies
        else:
            mid = (low + high )/ 2
            if k == self._table[mid]._key:
                return mid
            elif k < self._table[mid]._key:
                return self._find_index(k, low, mid - 1) # may return mid
            else:
                return self._find_index(k, mid + 1, high) # answer is right of mid
            
    #---------public behvaiors
    def __init__(self):
        """Create an empty map"""
        self._table = []
        
    def __len__(self):
        """Return the number of items in the map"""
        return len(self._table)
    
    def __getitem__(self, k):
        """Return value associated with key k, raise KeyError if not found"""
        j = self._find_index(k, 0, len(self._table) - 1)
        if j == len(self._table) or self._table[j]._key != k:
            raise KeyError('Key Error: ' + repr(k))
        return self._table[j]._value
    
    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present"""
        j = self._find_index(k, 0, len(self._table) - 1)
        if j < len(self._table) and self._table[j]._key == k:
            self._table[j]._value = v # reassign value
        else:
            self._table.insert(j, self._Item(k, v)) # create a new item and add it
            
    def __delitem__(self, k):
        """Remove item associated with key k. Raise KeyError if not found"""
        j = self._find_index(k, 0, len(self._table) - 1)
        if j == len(self._table) or self._table[j] != k:
            raise KeyError('Key Error: ' + repr(k))
        self._table.pop(j) # delete item
        
    def __iter__(self):
        """Generate keys of the map ordered from minimum to maximum"""
        for item in self._table:
            yield item._key
            
    def find_min(self):
        """Return (key, value) pair with minimum key (or None if empty)"""
        if len(self._table) > 0:
            return (self._table[0]._key, self._table[0]._value)
        else:
            return None
    
    def find_max(self):
        """Return (key, value) pair with maximum key (or None if empty)"""
        if len(self._table) > 0:
            return (self._table[-1]._key, self._table[-1]._value)
        else:
            return None
    
    def find_ge(self, k):
        """Return (key, value) pair with least key greater than or equal to k"""
        j = self._find_index(k, 0, len(self._table) - 1) # j's key >= k
        if j < len(self._table):
            return (self._table[j]._key, self._table[j]._value)
        else:
            return None
        
    def find_It(self, k):
        """Return (key, value) pair with greatest key strictly less than k"""
        j = self._find_index(k, 0, len(self._table) -1)
        if j > 0:
            return (self._table[j - 1]._key, self._table[j-1]._value)
        else:
            return None
        
    def find_gt(self, k):
        """Return (key, value) pair with least key strictly greater than k"""
        j = self._find_index(k, 0, len(self._table) - 1)
        if j < len(self._table) and self._table[j]._key == k:
            j += 1
        if j < len(self._table):
            return (self._table[j]._key, self._table[j]._value)
        else:
            return None
        
    def find_range(self, start, stop):
        """Iterate all (key, value) pairs such that start <= key < stop
        
        If start is None, iteration begings with minimum key of map
        If stop is None, iteration continues through the maximum key of map"""
        
        if start is None:
            j = 0
        else:
            j = self._find_index(start, 0, len(self._table) - 1) # find first result
            while j < len(self._table) and (stop is None or self._table[j]._key < stop):
                yield (self._table[j]._key, self._table[j]._value)
                j += 1